## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing:2px; color:#b57edc; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #b57edc">Libraries</p>

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm_notebook

import warnings
warnings.filterwarnings('ignore')

import re

pd.options.display.max_rows = 1000

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing:2px; color:#b57edc; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #b57edc">Intro</p>

도쿄의 **치안** 데이터와 **오시마랜드** 데이터를 활용하여 EDA(탐색적 데이터 분석)를 수행하고자 한다.

 <a href = 'http://www.oshimaland.co.jp'> 오시마랜드(Oshimaland)</a>
>This website showcases buildings with unfortunate past incidents such as suicide cases, corpse disposal incidents, and murder cases.

<a href = 'https://catalog.data.metro.tokyo.lg.jp/dataset?q=%E6%B2%BB%E5%AE%89&sort=score+desc%2C+metadata_modified+desc'>일본 도쿄 공공 데이터</a>

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing:2px; color:#b57edc; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #b57edc">EDA</p>

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing:2px; color:#068FFF; font-size:80%; text-align:left;padding: 0px; border-bottom: 3px solid #068FFF">Oshima Jieun Data Processing</p>

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing:2px; color:#1A5D1A; font-size:75%; text-align:left;padding: 0px; border-bottom: 3px solid #1A5D1A">Data info</p>

There are 4 independent variables:
<ul>
<li><strong>oshimaland_dataset_final.csv</strong><ul>
<br>
<li><code>info</code> 사건 정보</li>
<li><code>address</code> 사건이 발생한 주소</li>
<li><code>occurred_date</code> 사건 발생 날짜</li>
<li><code>recorede_date</code> 사건 기록 날짜</li>

In [ ]:
oshimaJ= pd.read_csv('/Users/genie/Documents/COLLABORATION/AirbnbWise/Oshimaland_data/jieun/oshimaland_dataset_final.csv')
oshimaJ.head()

In [ ]:
oshimaJ.drop(columns = ['recorded_date'], inplace = True) #* recorded_date(기록 날짜) 칼럼 제거
oshimaJ.rename(columns = {'occurred_date': 'date'}, inplace = True) #* occured_date -> date 칼럼명 변경

In [ ]:
#* address 칼럼 가공
#! 일본어 주소는 일반적으로 '시' 또는 '구', '동/읍/면', '번지', 그리고 '건물명' 등의 정보로 구성
#* (1) address의 모든 데이터에 '東京都(도쿄도)' 포함되어 있으므로 제거
oshimaJ['address'] = oshimaJ['address'].str.replace('東京都','')
oshimaJ.tail()

In [ ]:
#* (2) '시(市) / 구(区) 칼럼(shiku) 추가'
#* oshimaJ['address'].str.contains('市').sum(), 주소에 市(시)가 포함되어 있는 주소는 186개
#* 주소에 市(시)가 없다면 구(区)로 추출
def districtExtract(address):
    if '市' in address:
        extracted_data = address[:address.index('市') + 1]
    elif '市' not in address and '区' in address:
        extracted_data = address[:address.index('区') + 1]
    else: 
        extracted_data = address #* '外国外国板橋二丁目639' 데이터는 시, 구가 모두 없는 유일한 데이터로 삭제해준다.
    return extracted_data

oshimaJ['shiku'] = oshimaJ['address'].apply(districtExtract)

drop_index = oshimaJ[oshimaJ['shiku'] == '外国外国板橋二丁目639'].index
oshimaJ = oshimaJ.drop(drop_index).reset_index(drop=True)
display(oshimaJ.head(), oshimaJ.tail())

In [ ]:
#* (3) town(동/ 읍/ 면) 칼럼 추가 -> town 구역 + 건물 번호 모두 추출
shikuList = list(oshimaJ['shiku'])
def townExtract(address):
    if isinstance(address, str):
        for idx in range(len(shikuList)):
            if shikuList[idx] in address:
                find_data = address[address.find(shikuList[idx]) + len(shikuList[idx]):]
                return find_data

oshimaJ['town'] = oshimaJ['address'].apply(townExtract)
display(oshimaJ.head(), oshimaJ.tail())


In [ ]:
#* (4) 'town' 내에서 구역이 존재하는 데이터 들의 구역 추출하여 칼럼 추가
#* oshimaJ['town'].str.contains('丁目').sum(), '丁目' 이 존재하는 데이터는 542개, 존재하지 않는 데이터는 26개
def partExtract(town):
    if '丁目' in town:
        extracted_data = town[:town.find('丁目')] + '丁目'
    else:
        extracted_data = np.nan #* 구역 없음
    return extracted_data

oshimaJ['townpart'] = oshimaJ['town'].apply(partExtract)

In [ ]:
#* (5) 'town'내에서 구역이 존재하지 않는 데이터들의 상세 주소 추출하여 칼럼 추가
def detailExtract(town):
    if '丁目' not in town:
        extracted_data = town
    else:
        extracted_data = town[town.find('丁目') + 2:]
    return extracted_data 

oshimaJ['detailpart'] = oshimaJ['town'].apply(detailExtract)

In [ ]:
#* oshimaJ['town'].str.contains('丁目').sum(), '丁目' 이 존재하는 데이터는 542개
#* '丁目'이 존재하지 않는 경우, 주소에 숫자가 포함되어 있다면 숫자 이전 문자열 까지 구역으로 추출 하고, 숫자가 없다면 그대로 추출
#! 숫자가 두개 이상 포함된 경우가 있으므로, 이 함수는 적용하지 않는다.
# def partExtract(town):
#     if '丁目' in town:
#         extracted_data = town[:town.find('丁目')] + '丁目'
#     else:
#         for idx, char in enumerate(town):
#             if any(char.isdigit() for char in town): #* town 주소에서 하나 이상의 숫자가 발견되었는지 확인
#                 match = re.search(r'\d+', town)
#                 extracted_data = town[:match.start()]
#             else:
#                 extracted_data = town[idx]
#     return extracted_data

#oshimaJ['townpart'] = oshimaJ['town'].apply(partExtract)

In [ ]:
#* (4) 특수 문자 제거 -> 이 데이터의 경우 '\r', '\n', '↓' 를 제거해야 한다.
def remove_special_characters(text):
    if isinstance(text, str):  #* 문자열인 경우에만 정규표현식 적용
        pattern = r'[\r\n↓]'
        cleaned_text = re.sub(pattern, '', text)
        return cleaned_text
    return text

oshimaJ_cleaned = oshimaJ.applymap(remove_special_characters)

In [ ]:
oshimaJ_cleaned.shape

In [ ]:
#* (5) 칼럼 순서 변경
oshimaJ_cleaned = oshimaJ_cleaned.reindex(columns = ['info', 'date', 'address', 'shiku', 'town', 'townpart', 'detailpart'])

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing:2px; color:#068FFF; font-size:80%; text-align:left;padding: 0px; border-bottom: 3px solid #068FFF">Oshima Yunyoung Data Processing</p>

In [ ]:
oshimaY = pd.read_csv('/Users/genie/Documents/COLLABORATION/AirbnbWise/Oshimaland_data/yunyoung/selenium_tokyo_data.csv', index_col = 0)
oshimaY.head()

In [ ]:
#! 일본어 주소는 일반적으로 '시' 또는 '구', '동/읍/면', '번지', 그리고 '건물명' 등의 정보로 구성
#* (1) address의 모든 데이터에 '東京都(도쿄도)' 포함되어 있으므로 제거
oshimaY['address'] = oshimaY['address'].str.replace('東京都','')

In [ ]:
oshimaY.rename(columns = {'content': 'info', 'district' : 'shiku'}, inplace = True) #* content -> info 칼럼명 변경
oshimaY = oshimaY.reindex(columns = ['info', 'date','address', 'shiku'])
oshimaY.tail()

In [ ]:
#* oshimaJ_cleaned 데이터프레임에 맞춰 oshimaY 데이터프레임 변경
display(oshimaJ_cleaned.head(1), oshimaY.tail())

In [ ]:
#* (2) 'town' 칼럼 추가
shikuList = list(oshimaY['shiku'])
oshimaY['town'] = oshimaY['address'].apply(townExtract)

In [ ]:
#* (3) 'town' 칼럼 결측치를 처리해야 다음 코드에서 에러가 나지 않는다.
nullNum = oshimaY['town'].isnull().sum()
print(f'기존 결측치 개수 : {nullNum}개')
oshimaY['town'] = oshimaY['town'].fillna('Not data')

In [ ]:
#* (4) 'town' 내에서 구역이 존재하는 데이터 들의 구역 추출하여 칼럼 추가
oshimaY['townpart'] = oshimaY['town'].apply(partExtract)

In [ ]:
#* (5) 'town'내에서 구역이 존재하지 않는 데이터들의 상세 주소 추출하여 칼럼 추가
oshimaY['detailpart'] = oshimaY['town'].apply(detailExtract)

In [ ]:
#* (6) 특수문자 제거 
oshimaY_cleaned = oshimaY.applymap(remove_special_characters)

In [ ]:
#* (7) oshimaY 데이터 중 None 값을 NaN으로 대체
oshimaY_cleaned = oshimaY_cleaned.applymap(lambda x: np.nan if x is None else x)

In [ ]:
#* (8) oshimaY town 관련 데이터 중 'Not data' 값을 NaN으로 대체
oshimaY_cleaned = oshimaY_cleaned.applymap(lambda x: np.nan if x == 'Not data' else x)
oshimaY_cleaned.head()

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing:2px; color:#068FFF; font-size:80%; text-align:left;padding: 0px; border-bottom: 3px solid #068FFF">Data 탐색</p>

In [ ]:
#* 각 데이터 shape
print(oshimaJ_cleaned.shape, oshimaY_cleaned.shape)

In [ ]:
#* 각 데이터 결측치 확인
print(oshimaJ_cleaned.isnull().sum())
print('-' * 15)
print(oshimaY_cleaned.isnull().sum())

In [ ]:
#* 각 데이터 중복데이터 확인
print(oshimaJ_cleaned.duplicated().sum(), oshimaY_cleaned.duplicated().sum())

In [ ]:
#* 두 데이터 병합
oshima = pd.concat([oshimaJ_cleaned, oshimaY_cleaned]).reset_index(drop = True)
print(oshima.shape)

In [ ]:
#* town 칼럼 삭제
oshima = oshima.drop(columns = ['town'])

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing:2px; color:#1A5D1A; font-size:75%; text-align:left;padding: 0px; border-bottom: 3px solid #1A5D1A">Data info</p>

There are 6 independent variables:
<ul>
<li><strong>oshima</strong><ul>
<br>
<li><code>info</code> 사건 정보</li>
<li><code>date</code> 사건 발생 날짜</li>
<li><code>address</code> 사건이 발생한 주소</li>
<li><code>shiku</code> 시/구</li>
<li><code>townpart</code> 동/읍/면</li>
<li><code>detailpart</code> 상세 주소</li>

In [ ]:
#* 중복 데이터 제거
oshima = oshima.drop_duplicates()
oshima = oshima.reset_index(drop=True)
print(oshima.shape)

In [ ]:
oshima.isnull().sum() #! 결측치 처리 고민...

In [ ]:
oshima.to_csv('oshimaEDA.csv', index = False)

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing:2px; color:#068FFF; font-size:80%; text-align:left;padding: 0px; border-bottom: 3px solid #068FFF">Tokyo Open Data Processing</p>

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing:2px; color:#1A5D1A; font-size:75%; text-align:left;padding: 0px; border-bottom: 3px solid #1A5D1A">Data info</p>

There are 20 independent variables:
<ul>
<li><strong>TokyoCrime.csv</strong><ul>
<br>
<li><code>罪名</code> 범죄의 종류 또는 범죄명</li>
<li><code>手口</code> 범죄의 접근 방식 또는 행동</li>
<li><code>管轄警察署（発生地）</code> 범죄가 발생한 지역의 관할 경찰서</li>
<li><code>管轄交番・駐在所（発生地）</code> 범죄가 발생한 지역의 관할 파출소 또는 장애소</li>
<li><code>市区町村コード（発生地）</code> 범죄가 발생한 지역의 시, 구, 동 등을 식별하는 코드</li>
<li><code>都道府県（発生地）</code> 범죄가 발생한 지역의 도 또는 특별시</li>
<li><code>市区町村（発生地）</code> 범죄가 발생한 지역의 시, 구, 동</li>
<li><code>町丁目（発生地）</code> 범죄가 발생한 지역의 동 등 자세한 정보</li>
<li><code>発生時（始期）</code> 범죄가 발생한 날짜</li>
<li><code>発生年月日（始期）</code> 범죄가 발생한 시간</li>
<li><code>発生場所の属性</code> 범죄가 발생한 장소의 특성</li>
<li><code>現金以外の主な被害品</code> 현금 이외의 주요 피해 물품</li>
<li><code>被害者の性別</code> 피해자의 성별</li>
<li><code>被害者の年齢</code>  피해자의 나이</li>
<li><code>現金被害の有無</code> 현금 피해의 여부</li>
<li><code>施錠関係</code> 잠금 관련 정보</li>
<li><code>盗難防止装置の有無</code> 도난 방지 장치의 여부</li>
<li><code>被害者の職業</code> 피해자의 직업</li>
<li><code>発生場所</code> 범죄가 발생한 장소</li>
<li><code>発生場所の詳細</code> 범죄가 발생한 장소에 대한 자세한 정보</li>


In [ ]:
openData = pd.read_csv('/Users/genie/Documents/COLLABORATION/AirbnbWise/Tokyo_OpenData/Integration of downloaded CSV files/TokyoCrime.csv',  encoding='SHIFT_JIS', index_col=0)
openData.shape

In [ ]:
#* : 칼럼명을 영어로 변환
#TODO : 칼럼명이 영어 표기와 기존 일본어 표기 중 어떤 것이 더 나은지 고민이다.
openData.columns = ['crimeType', 'criminalBehavior', 'policeStation', 'policeboxOrsubstation','crimeCode', 
                     'crimeAddressSido', 'crimeAddressDong', 'crimeDetailAddress', 'dateOfCrime', 'timeOfCrime', 'crimeSceneCharacteristics', 
                     'otherThanCash', 'genderOfVictim', 'ageOfVictim', 'cashDamagePresence', 'lockingInfo','preventionDevicePresence',
                     'jobOfVictim', 'crimeScene', 'crimeDetailinfo']

In [ ]:
openData.info() 

In [ ]:
#* 칼럼 중 '被害者の性別'(genderOfVictim) - 전체 데이터에서 0.5%
# '盗難防止装置の有無'(preventionDevicePresence) - (전체 데이터의 3%)
#  두 칼럼은 압도적으로 데이터가 적으므로 삭제한다. 
#TODO '現金以外の主な被害品'(otherThanCash 11039 Not Null) 칼럼은 전체 데이터의 8%, '現金被害の有無'(cashDamagePresence 11200 Not Null) 칼럼도 전체 데이터의 8% 이다. 이 두 칼럼은 삭제할지 말지 고민이다.
openData = openData.drop(columns = ['genderOfVictim', 'preventionDevicePresence'])
openData.shape

In [ ]:
# TODO: 'timeOfCrime' 칼럼 데이터 시간 양식이 동일하지 않다. 양식을 통일 하고, 데이터 타입을 datetime(ns)로 변경한다.
# ex) 2023-07-18 , 2023/07/18
# 추후 year, month, day 칼럼으로 분리하여 추가할 예정이다.
def getDateOfCrime(date):
    if '-' in date:
        dateValue = pd.to_datetime(date, format='%Y-%m-%d')
    elif '/' in date:
        dateValue = pd.to_datetime(date, format='%Y-%m-%d')
    else: 
        dateValue = pd.to_datetime(date, errors = 'coerce') #* 날짜가 잘못된 경우는 Nat(Not a Time)으로 처리
    return dateValue

openData['dateOfCrime'] = openData['dateOfCrime'].apply(getDateOfCrime)

In [ ]:
# TODO : 이 데이터에서 'crimeType' 즉, 범죄 유형은 '窃盗'(절도)만 존재 하므로, 'crimeType' 칼럼은 삭제한다.
print(openData['crimeType'].value_counts())
openData = openData.drop(columns = ['crimeType'])

In [ ]:
# TODO 범죄 주소 시/도 + 동/읍/면 + 상세 주소 텍스트를 합쳐서 전체 주소 칼럼을 만들어서, oshimaland 데이터의 주소값과 비교할 수 있도록 한다.
openData['crimeFullAddress'] = openData['crimeAddressSido'] + openData['crimeAddressDong'] + openData['crimeDetailAddress']
display(oshima.head(), display(openData.head()) )
#* oshima 데이터의 'townpart' 칼럼과 openData 데이터의 'crimeDetailAddrss' 칼럼 모두, 구역을 표현한다.